In [1]:
import pytorch_lightning as pl
from transformers import AutoTokenizer

import torch
from torch import nn

import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger

from iterativenn.utils.data_factory import DataFactory
from iterativenn.utils.DataModules import LMDataModule
from iterativenn.nn_modules.Sequential2D import Sequential2D
from iterativenn.lit_modules import IteratedModelBatch
from torch.utils.data import DataLoader
import logging
import warnings
import torchtext
import datasets
from transformers import (
    AutoModelForMaskedLM,
    AutoTokenizer,
    AutoConfig,
)
from transformers.optimization import AdamW
import math
import os
import warnings
from dataclasses import dataclass
from typing import List, Optional, Tuple, Union

import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

/Users/hpathak/opt/anaconda3/envs/res/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [4]:
global_max_epochs = 1
global_optimizer = 'SGD'
model_name_or_path="bert-base-cased"
config = AutoConfig.from_pretrained(
            model_name_or_path, return_dict=True)

output_len = config.vocab_size

cfg = {
    "sequential2D": {
        "in_features_list": [768, 10, output_len], 
        "out_features_list": [768, 10, output_len], 


        "block_types": [
            [None, 'Linear', None],
            [None, None, 'Linear'],
            [None, None, None],
        ],
        "block_kwargs": [
            [None, None, None],
            [None, None, None],
            [None, None, None],
        ]
    },
    "callbacks": {
        "loss": {
            "func": "CrossEntropyLoss",
            "idx_list" : range(768+10, 768+10+output_len),
            "sequence_position": 'all',
        },
        "initialization": {
            "func": "zeros",
            "size":768+10+output_len,
            "seq_len": 3,
            "batch_size": 4,
        },
        "data": {
            "func": "insert",
            "idx_list": range(768),
            "flatten_input": False,            
        },
        "output": {
            "func": "max",
            "idx_list" : range(768+10, 768+10+output_len)
        },
    }
}




In [5]:
data_module = LMDataModule(
        model_name_or_path='distilbert-base-cased',
        line_by_line=False,
        pad_to_max_length=False,
        preprocessing_num_workers=1,
        overwrite_cache=False,
        max_seq_length=3,
        mlm_probability=0.15,
        train_batch_size=4,
        val_batch_size=4,
        dataloader_num_workers=8,
    )


logger = TensorBoardLogger("outputs", name="nlp-demo", version='main')


sequential2D = Sequential2D.from_config(cfg["sequential2D"])
callbacks = IteratedModelBatch.ConfigCallbacks(cfg["callbacks"])
model = IteratedModelBatch.IteratedModelNLP(sequential2D, callbacks, optimizer=global_optimizer)

torch.autograd.set_detect_anomaly(True)
trainer = Trainer(
        accelerator='auto',
        devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
        max_epochs=global_max_epochs,
        log_every_n_steps=1,
        enable_progress_bar=False,
        logger=logger)

trainer.fit(model, data_module)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


No optimization routine provided


Found cached dataset wikitext (/Users/hpathak/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/hpathak/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-b8fb876458051540.arrow
Loading cached processed dataset at /Users/hpathak/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-6e4784073618cb72.arrow
Loading cached processed dataset at /Users/hpathak/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-8aa87176c76c40ea.arrow


...

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

/Users/hpathak/opt/anaconda3/envs/res/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory outputs/nlp-demo/main/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name  | Type         | Params
---------------------------------------
0 | model | Sequential2D | 326 K 
---------------------------------------
326 K     Trainable params
0         Non-trainable params
326 K     Total params
1.307     Total estimated model params size (MB)
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` m

Accuracy: nan
Accuracy: 0.0


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with 